#  项目分析：
- 构成：
    - 蛇 Snake
    - 食物 Food
    - 世界 World
    
- 蛇和食物属于整个世界
    
        class World:
            self.snake
            self.food
    - 上面代码不太友好
    - 我们用另外一个思路来分析
    
- 我们的分析思路
    - 食物是一个独立的事物
    - 蛇也可以认为是一个独立的事物
    - 世界也是，但世界负责显示

In [1]:
import random
class Food():
    '''
    功能：
        1. 出现在画面的某一个地方
        2. 一旦被吃，则增加蛇的分数

    '''
    def __init__(self,queue):
        '''
        自动产生一个食物
        '''
        self.queue = queue
        self.new_food()
        
    def new_food(self):
        '''
        功能：产生一个食物
        产生一个食物的过程就是随机产生一个食物坐标的过程
        '''
        # 注意横纵坐标产生的范围
        x = random.randrange(50,480,10)
        y = random.randrange(30,280,10)
        # 同理产生y坐标
        # 需要注意的是，我们的正给游戏屏幕一般不需要把他设置成正方形
        
        
        self.postion = x,y # position存放食物的位置
        
        # 队列，就是一个不能够随意访问内部元素，只能从头弹出一个元素并只能
        #从队尾追加元素的list
        #  把一个食物产生的消息放入队列
        # 消息的格式，自己定义
        # 我的定义是： 消息是一个dict， k代表消息类型，v代表此类型的数据
        self.queue.put({"food": self.postion})

In [2]:
import threading
class Snake(threading.Thread):
    '''
    蛇的功能：
        1. 蛇能动，由我们的上下左右按键控制
        2. 蛇每次动，都需要重新计算蛇头的位置
        3. 检测是否游戏完事的功能
    '''
    def __init__(self, world, queue):
        threading.Thread.__init__(self)
        
        self.world = world
        self.queue=queue
        self.points_earned = 0 #游戏分数
        self.food = Food(self.queue)
        self.snake_points = [(490,50),(480,50),(460,50),(450,50)]
        self.direction ='Left'
        
        self.start()
        
        
    def run(self):
        '''
        一旦启用多线程调用此函数
        要求蛇一直都在跑,检查是否和食物相遇
        '''
        if self.world.is_game_over:
            self._delete()
        
        while not self.world.is_game_over:
            self.queue.put({"move": self.snake_points})
            time.sleep(0.5) #控制蛇的速度
            self.move()
        
        
    
    def move(self):
        '''
        负责蛇的移动
        1. 重新计算蛇头的坐标
        2. 当蛇头跟食物相遇，则加分，重新生成食物，通知world，加分
        3. 否则, 蛇需要动
        '''
        new_snake_point = self.cal_new_position() #重新计算蛇头位置
        
        # 蛇头位置跟食物位置相同
        if self.food.postion == new_snake_point:
            print("goodddddd")
            self.points_earned += 1 #得分加1
            self.queue.put({"points_earned": self.points_earned})
            self.food.new_food() #就得食物被吃掉，产生新的食物
        else:
            # 需要注意蛇的信息的保存方式
            # 每次移动是删除存放蛇的最前位置，并在后面追加
            print(new_snake_point)
            print(self.food.postion)
            self.snake_points.pop(0)
            # 判断程序是否退出，因为新的蛇可能撞墙
            self.check_game_over(new_snake_point)
            self.snake_points.append(new_snake_point)
    def cal_new_position(self): 
        '''
        计算新的 蛇头的位置
        '''
        last_x, last_y = self.snake_points[-1]
        if self.direction == "Up": #direction负责存储蛇移动的方向
            new_snake_point = last_x, last_y - 10 #每次移动的跨度是10像素
        elif self.direction == 'Down':
            # 需要总共判断上下左右四个方向
             new_snake_point = last_x, last_y + 10
        elif self.direction == 'Left':          
            new_snake_point = last_x -10, last_y 
                
        elif self.direction == 'Right':           
             new_snake_point = last_x+10, last_y 
        return new_snake_point
        
    def key_pressed(self, e):
        # keysym是按键名称
        self.direction = e.keysym
        
    def check_game_over(self, snake_point):
        '''
        判断的依据是蛇头是否和墙相撞
        '''
        # 把蛇头的坐标拿出来，跟墙的坐标进行判断
        x,y = snake_point[0], snake_point[1]
        if not -5 < x < 605 or not -5 < y < 405 or snake_point in self.snake_points:
            self.queue.put({'game_over': True})


In [3]:
from tkinter import *
from  tkinter import messagebox

import queue
import time
class World(Tk):
    '''
    用来模拟整个游戏画板
    '''
    def __init__(self,q):
        Tk.__init__(self)
        
        self.queue =q
        self.is_game_over = False
        
        # 定义画板
        self.canvas = Canvas(self, width=600, height=400, bg='black')
        self.canvas.pack()
        
        
        # 画出蛇和食物
        self.snake = self.canvas.create_line((0,0), (0,0), fill="white", width=10)
        #self.food = self.canvas.create_rectangle(0,0,0,0,fill='white', outline='white')

        self.points_earned = self.canvas.create_text(450, 20,fill='white', text='SCORE: 0')
        self.queue_handler()
  

    def queue_handler(self):
        try:
            # 需要不断从消息队列拿到消息，所以使用死循环
            while True:
                task = self.queue.get(block=False)
            
                if task.get("game_over"):
                    self.game_over()
                if task.get("move"):
                    
                    points =  [ x for point in task['move'] for x in point]
                    #重新绘制蛇
                    self.canvas.coords(self.snake, *points)
                if task.get("food"):
                    x,y=task['food'][0],task['food'][1]
                    self.food = self.canvas.create_rectangle(x,y,x+10,y+10,fill='white', outline='white')
                
                
                if task.get("points_earned"):
                    print("yes")
                    self.canvas.delete(self.food)
                    self.canvas.delete(self.points_earned)
                    self.points_earned = self.canvas.create_text(450, 20,fill='white', text='SCORE: {0}'.format(task['points_earned']))
               
                
        except queue.Empty: #爆出队列为空异常
            if not  self.is_game_over:
                # after的含义是，在多少毫秒后调用后面的函数
                self.canvas.after(100, self.queue_handler)
    
    def game_over(self):
        '''
        游戏结束，清理现场
        '''
        self.is_game_over = True
        #self.canvas.create_text("Game Over")
        
        #qb = Button(self, text="Quit", command=self.destroy)
        #rb = Button(self, text="Again", command=self.__init__)
        res = messagebox.askquestion(title="Game Over", message="You lose,try again.")       
        if res =='yes':
            self.is_game_over = False
            self.__init__(q)
        else :
            self.destroy()
        
        
        
        
if __name__ == "__main__":
    q = queue.Queue()
    world = World(q)
    
    snake = Snake(world, q)
    
    world.bind('<Key-Left>', snake.key_pressed)
    world.bind('<Key-Right>', snake.key_pressed)
    world.bind('<Key-Up>', snake.key_pressed)
    world.bind('<Key-Down>', snake.key_pressed)
    # 同样绑定右键，上下键
    
    world.mainloop()
                
                
                
                
                
                
                
                

(440, 50)
(170, 30)
(430, 50)
(170, 30)
(420, 50)
(170, 30)
(410, 50)
(170, 30)
(400, 50)
(170, 30)
(390, 50)
(170, 30)
(390, 40)
(170, 30)
(390, 30)
(170, 30)
(380, 30)
(170, 30)
(370, 30)
(170, 30)
(360, 30)
(170, 30)
(350, 30)
(170, 30)
(340, 30)
(170, 30)
(330, 30)
(170, 30)
(320, 30)
(170, 30)
(310, 30)
(170, 30)
(300, 30)
(170, 30)
(290, 30)
(170, 30)
(280, 30)
(170, 30)
(270, 30)
(170, 30)
(260, 30)
(170, 30)
(250, 30)
(170, 30)
(240, 30)
(170, 30)
(230, 30)
(170, 30)
(220, 30)
(170, 30)
(210, 30)
(170, 30)
(200, 30)
(170, 30)
(190, 30)
(170, 30)
(180, 30)
(170, 30)
goodddddd
yes
(170, 30)
(190, 240)
(160, 30)
(190, 240)
(150, 30)
(190, 240)
(140, 30)
(190, 240)
(140, 40)
(190, 240)
(140, 50)
(190, 240)
(140, 60)
(190, 240)
(140, 70)
(190, 240)
(140, 80)
(190, 240)
(140, 90)
(190, 240)
(140, 100)
(190, 240)
(140, 110)
(190, 240)
(140, 120)
(190, 240)
(140, 130)
(190, 240)
(140, 140)
(190, 240)
(140, 150)
(190, 240)
(140, 160)
(190, 240)
(140, 170)
(190, 240)
(140, 180)
(190, 240

In [4]:
#help (Tk)

(100, 220)
(220, 190)
(90, 220)
(220, 190)
(80, 220)
(220, 190)
(70, 220)
(220, 190)
(60, 220)
(220, 190)
(50, 220)
(220, 190)
